# CrewAI Callbacks: Monitoring and Control

Callbacks allow you to monitor agent execution and track progress in real-time.

## 1. Setup

In [ ]:
# !pip install crewai crewai-tools langchain-openai python-dotenv

In [ ]:
import os
import json
import time
from datetime import datetime
from crewai import Agent, Task, Crew, Process, LLM
from crewai.tools import tool


# Set your API key
# os.environ["OPENAI_API_KEY"] = "your-api-key"

os.environ["OPENAI_API_KEY"] = os.getenv("OPEN_ROUTER_KEY")
os.environ["SERPER_API_KEY"] = os.getenv("SERPER_API_KEY")
os.environ['LITELLM_LOG'] = 'DEBUG' 
os.environ['OPENAI_API_BASE'] = 'https://openrouter.ai/api/v1'
os.environ['OPENAI_BASE_URL'] = 'https://openrouter.ai/api/v1'


# Initialize the language model
llm = LLM(
        model='openai/gpt-4o',
        api_key=os.getenv('OPEN_ROUTER_KEY'),
        base_url="https://openrouter.ai/api/v1"
    )

# Global storage for callback data (in production, use proper storage)
callback_logs = []
performance_metrics = {}
# Storage for logs
callback_logs = []

## 2. Basic Callbacks

### Important: Callback Signatures

The exact callback signatures can vary by CrewAI version. Common patterns:
- `task_callback(output)` - Called when a task completes
- `step_callback(step_output)` - Called for each step (may receive different object types)

In [ ]:
# Simple task callback
def task_callback(output):
    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    print(f"[{timestamp}] Task completed!")
    print(f"Output length: {len(str(output))} characters")
    print("-" * 50)

# Step callback - Fixed to handle different parameter types
def step_callback(step_output):
    """Step callback that handles the output from each step."""
    timestamp = datetime.now().strftime('%H:%M:%S')
    print(f"[{timestamp}] Step executed")
    
    # Check what type of output we received
    output_type = type(step_output).__name__
    print(f"Output type: {output_type}")
    
    # Show preview of output
    if hasattr(step_output, '__dict__'):
        print(f"Attributes: {list(step_output.__dict__.keys())[:5]}...")
    else:
        print(f"Output preview: {str(step_output)[:100]}...")

# Safe wrapper for any callback - DEFINE THIS FIRST
def make_safe_callback(callback_func):
    """Wraps any callback to handle errors gracefully."""
    def safe_wrapper(*args, **kwargs):
        try:
            return callback_func(*args, **kwargs)
        except Exception as e:
            print(f"Callback error handled: {type(e).__name__}: {e}")
            print(f"Continuing execution...")
    return safe_wrapper

# Flexible step callback that handles different parameter counts
def flexible_step_callback(*args, **kwargs):
    """Step callback that adapts to different signatures."""
    timestamp = datetime.now().strftime('%H:%M:%S')
    print(f"[{timestamp}] Step callback triggered")
    print(f"Args received: {len(args)}")
    print(f"Kwargs received: {list(kwargs.keys())}")
    
    # Try to extract useful information
    for i, arg in enumerate(args):
        arg_type = type(arg).__name__
        print(f"  Arg {i}: {arg_type}")
        
        # If it has a role attribute, it might be an agent
        if hasattr(arg, 'role'):
            print(f"    - Role: {arg.role}")
        
        # If it's a string or has string representation
        if isinstance(arg, str) or hasattr(arg, '__str__'):
            preview = str(arg)[:50]
            print(f"    - Preview: {preview}...")

## 3. Create Tools (with proper docstrings)

In [ ]:
# Tool functions MUST have docstrings and type hints
def search_func(query: str) -> str:
    """Search for information."""
    return f"Found information about: {query}"

def analyze_func(data: str) -> str:
    """Analyze data."""
    return f"Analysis of: {data}"

# Create tools
search_tool = tool("Search")(search_func)
analyze_tool = tool("Analyze")(analyze_func)

## 4. Create Agents and Tasks

In [ ]:
# Create agents
researcher = Agent(
    role="Researcher",
    goal="Find information",
    backstory="Expert researcher",
    tools=[search_tool],
    llm=llm
)

analyst = Agent(
    role="Analyst",
    goal="Analyze findings",
    backstory="Expert analyst",
    tools=[analyze_tool],
    llm=llm
)

# Create tasks
research_task = Task(
    description="Research AI trends",
    expected_output="Research findings",
    agent=researcher
)

analysis_task = Task(
    description="Analyze the research",
    expected_output="Analysis report",
    agent=analyst
)

## 5. Create Crew with Callbacks

In [ ]:
# Create crew with callbacks
crew = Crew(
    agents=[researcher, analyst],
    tasks=[research_task, analysis_task],
    process=Process.sequential,
    task_callback=task_callback,
    step_callback=step_callback,  # Using the fixed step_callback
    verbose=True
)

print("Crew created with callbacks!")

# Alternative: Create crew with flexible callbacks
flexible_crew = Crew(
    agents=[researcher, analyst],
    tasks=[research_task, analysis_task],
    process=Process.sequential,
    task_callback=make_safe_callback(task_callback),
    step_callback=make_safe_callback(flexible_step_callback),
    verbose=True
)

print("Alternative crew created with flexible callbacks!")

## 6. Execute the Crew

In [ ]:
# Execute crew
print("Starting crew execution...")
print("=" * 50)

start_time = time.time()

# Try the standard crew first
try:
    result = crew.kickoff()
    print("Standard crew executed successfully!")
except Exception as e:
    print(f"Standard crew failed: {e}")
    print("Trying flexible crew...")
    result = flexible_crew.kickoff()

end_time = time.time()

print("=" * 50)
print(f"Execution time: {end_time - start_time:.2f} seconds")
print(f"Result preview: {str(result)[:200]}...")

## 7. Advanced Callbacks

In [ ]:
# Performance monitoring
task_times = {}

def performance_callback(output):
    task_id = id(output)
    if task_id not in task_times:
        task_times[task_id] = time.time()
    else:
        duration = time.time() - task_times[task_id]
        print(f"Task completed in {duration:.2f}s")

# Data collector
def collect_data(output):
    global callback_logs
    log = {
        'timestamp': datetime.now().isoformat(),
        'output': str(output)[:100]
    }
    callback_logs.append(log)
    print(f"Logged data (total logs: {len(callback_logs)})")

## 8. Callback Factory Functions

In [ ]:
# Create custom callbacks
def create_logger(level="INFO"):
    def logger(output):
        timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        print(f"[{level}] [{timestamp}] {str(output)[:50]}...")
    return logger

# Create filter callback
def create_filter(keyword):
    def filter_callback(output):
        if keyword.lower() in str(output).lower():
            print(f"Found '{keyword}' in output!")
    return filter_callback

# Test factory functions
info_logger = create_logger("INFO")
error_filter = create_filter("error")

info_logger("Test message")
error_filter("No error here")
error_filter("An error occurred")

## 9. Combining Callbacks

In [ ]:
# Combine multiple callbacks
def combine_callbacks(*callbacks):
    def combined(output):
        for cb in callbacks:
            try:
                cb(output)
            except Exception as e:
                print(f"Error in callback: {e}")
    return combined

# Create combined callback
multi_callback = combine_callbacks(
    task_callback,
    performance_callback,
    collect_data,
    create_logger("DEBUG")
)

# Use with crew
multi_crew = Crew(
    agents=[researcher, analyst],
    tasks=[research_task, analysis_task],
    process=Process.sequential,
    task_callback=multi_callback,
    verbose=True
)

## 10. Dashboard Example

In [ ]:
# Create dashboard
def create_dashboard():
    stats = {'tasks': 0, 'start': time.time()}
    
    def dashboard(output):
        stats['tasks'] += 1
        elapsed = time.time() - stats['start']
        print(f"\n{'='*40}")
        print(f"DASHBOARD")
        print(f"Tasks: {stats['tasks']}")
        print(f"Time: {elapsed:.1f}s")
        print(f"{'='*40}\n")
    
    return dashboard

# Create and test dashboard
dashboard = create_dashboard()
dashboard("Test output 1")
time.sleep(1)
dashboard("Test output 2")

## 11. Error Handling

In [ ]:
# Safe callback wrapper
def safe_callback(callback_func):
    def wrapper(*args, **kwargs):
        try:
            return callback_func(*args, **kwargs)
        except Exception as e:
            print(f"Callback error: {e}")
    return wrapper

# Error tracking
error_count = 0

def error_tracker(output):
    global error_count
    if "error" in str(output).lower():
        error_count += 1
        print(f"Error detected! Total: {error_count}")

# Make it safe
safe_error_tracker = safe_callback(error_tracker)

# Test
safe_error_tracker("No problem")
safe_error_tracker("Error found")

## 12. Save Logs

In [ ]:
# Save logs to file
def save_logs(filename="logs.json"):
    def saver(output):
        log = {
            'time': datetime.now().isoformat(),
            'data': str(output)[:200]
        }
        try:
            with open(filename, 'a') as f:
                json.dump(log, f)
                f.write('\n')
            print(f"Saved to {filename}")
        except Exception as e:
            print(f"Save error: {e}")
    return saver

# Analyze logs
def analyze_logs():
    if not callback_logs:
        print("No logs")
        return
    
    print(f"Total logs: {len(callback_logs)}")
    for i, log in enumerate(callback_logs[-5:]):
        print(f"{i+1}. {log['timestamp']}: {log['output']}")

## Troubleshooting Callback Errors

### Common Issues:

1. **'ToolResult' object has no attribute 'role'**
   - The step_callback may receive different object types
   - Use flexible callbacks that check object types

2. **Wrong number of arguments**
   - Callback signatures can vary by CrewAI version
   - Use `*args, **kwargs` for flexibility

3. **Callbacks breaking execution**
   - Always wrap callbacks in error handlers
   - Use the `make_safe_callback` wrapper

### Debug Helper:

In [ ]:
# Debug callback to understand what parameters are being passed
def debug_callback(*args, **kwargs):
    """Use this to debug what parameters callbacks receive."""
    print("\n=== DEBUG CALLBACK ===")
    print(f"Number of args: {len(args)}")
    print(f"Number of kwargs: {len(kwargs)}")
    
    for i, arg in enumerate(args):
        print(f"\nArg {i}:")
        print(f"  Type: {type(arg)}")
        print(f"  Value preview: {str(arg)[:100]}...")
        print(f"  Attributes: {dir(arg)[:10]}...")
    
    for key, value in kwargs.items():
        print(f"\nKwarg '{key}':")
        print(f"  Type: {type(value)}")
        print(f"  Value: {str(value)[:100]}...")
    
    print("\n=== END DEBUG ===")

# Use debug callbacks to understand the signature
debug_crew = Crew(
    agents=[researcher],
    tasks=[research_task],
    process=Process.sequential,
    task_callback=debug_callback,
    step_callback=debug_callback,
    verbose=False  # Reduce noise for debugging
)

print("Debug crew created - run it to see callback parameters")